In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
import time
import gc
from tqdm import tqdm

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
import lightgbm as lgb

In [4]:
# label = pd.DataFrame()

# for i in tqdm(range(10)):
#     train_sample = pd.read_csv('./output_r2/train_kflod' + str(i), usecols=['aid', 'uid', 'label'])
#     train_sample['label'][train_sample.label == -1] = 0
#     label = pd.concat([label, train_sample])

# label.shape
# label.to_csv('./output_r2/train_label.csv', index=None)

(45539700, 3)

In [6]:
train = np.load('./output_r2/train_new.npy')
label = pd.read_csv('./output_r2/train_label.csv', usecols=['label'])

train.shape
label.shape

(20000000, 185)

(11727304, 185)

(20000000, 1)

In [4]:
train = train[:20000000, :]
label = label.iloc[:20000000, :]
gc.collect()

11

In [4]:
train_len = 45539700
test_len = 11729073

In [18]:
# # lgb_cv
# params = {
#     'objective':'binary',
# #     'metric':{ 'auc'},
#     'num_leaves':7,
#     'max_depth':3,
#     'learning_rate':0.05,
#     'reg_alpha' : .4,
#     'reg_lambda' : .2,
#     'random_state':1024,
#     'colsample_bytree':0.8,
#     'subsample':0.9,
# #     'n_estimators':20000,
#     'n_jobs': 31,
# #     'device':'gpu',
#     'histogram_pool_size':30270,
#     'max_bin':63,
#     'gpu_use_dp':True
# }

# lgb_train = lgb.Dataset(train, label=by['label'])
# print('Start training')
# cv = lgb.cv(params, lgb_train, num_boost_round=10000, nfold=3, early_stopping_rounds=50, seed=1024, verbose_eval=100)
# print('Baseline->' + str(len(cv['binary_logloss-mean'])) + ':' + str(cv['binary_logloss-mean'][-1]))

Start training


KeyboardInterrupt: 

In [8]:
# CJF新参数cv
cv_params = {
    'learning_rate':0.1,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 2**6,
    'max_depth':4,
    'min_child_weight':50, 
    'feature_fraction': 0.7,
    'bagging_fraction': 0.7,
    'lambda_l1':0,
    'lambda_l2':1,
    'histogram_pool_size':30270,
    'max_bin':63,
    'random_state':2018
}

lgb_train = lgb.Dataset(train, label=label['label'])
print('Start training')
cv = lgb.cv(cv_params, lgb_train, num_boost_round=10000, nfold=3, early_stopping_rounds=50, seed=1024, verbose_eval=100)
print('Baseline->' + str(len(cv['auc-mean'])) + ':' + str(cv['auc-mean'][-1]))

Start training
[100]	cv_agg's auc: 0.685207 + 0.000333392
[200]	cv_agg's auc: 0.695925 + 0.000536471
[300]	cv_agg's auc: 0.700486 + 0.000387777
[400]	cv_agg's auc: 0.703287 + 0.000454333
[500]	cv_agg's auc: 0.705152 + 0.000520955
[600]	cv_agg's auc: 0.706576 + 0.000506239
[700]	cv_agg's auc: 0.707736 + 0.000433519
[800]	cv_agg's auc: 0.708784 + 0.000495962
[900]	cv_agg's auc: 0.709625 + 0.000593563
[1000]	cv_agg's auc: 0.710295 + 0.000578106
[1100]	cv_agg's auc: 0.710886 + 0.000568602
[1200]	cv_agg's auc: 0.711396 + 0.000549444
[1300]	cv_agg's auc: 0.711902 + 0.000541909
[1400]	cv_agg's auc: 0.712293 + 0.000526453
[1500]	cv_agg's auc: 0.712702 + 0.000553686
[1600]	cv_agg's auc: 0.713047 + 0.000541239
[1700]	cv_agg's auc: 0.713348 + 0.000531621
[1800]	cv_agg's auc: 0.713663 + 0.000507218
[1900]	cv_agg's auc: 0.713947 + 0.000542918
[2000]	cv_agg's auc: 0.71416 + 0.000552524


KeyboardInterrupt: 

In [5]:
lgb_online = lgb.LGBMClassifier(
        objective='binary',
        boosting_type='gbdt',
        metric='auc',
        num_leaves=2**6,#35,
        max_depth=4,
        learning_rate=0.05, # small_eta
        min_child_weight=50,
        feature_fraction=0.7,
        bagging_fraction=0.7,
        lambda_l1=0,
        lambda_l2=1,
        random_state=2018,
        n_estimators=2000,
        n_jobs = 31,
        histogram_pool_size=30270,
        max_bin=63
    )

submit_model = lgb_online.fit(train, label['label'])

# test['score'] = submit_model.predict_proba(df_all[-test_len:, :])[:,1]
# test['score'] = test['score'].apply(lambda x: float('%.8f' % x))
# sub_zip_name = 'lgb_r2'
# test[['aid', 'uid','score']].to_csv('submission.csv', index=False)
# os.system('zip -j '+sub_zip_name+'.zip submission.csv')
# os.system('mv submission.csv ./submit/r2/'+sub_zip_name+'.csv')
# os.system('mv '+sub_zip_name+'.zip ./submit/r2/')

In [7]:
# test = np.load('./output_r2/test_new_b.npy')

test.shape

(11727304, 185)

In [8]:
score = submit_model.predict_proba(test)[:,1]

In [10]:
test = pd.read_csv('../../tencent_ad/CJY/T_r2/data/ori_data/test2.csv', usecols=['aid', 'uid'])

test.shape

(11727304, 2)

In [11]:
import os
test['score'] = score
test['score'] = test['score'].apply(lambda x: float('%.8f' % x))
sub_zip_name = 'lgb_zero_r2_b'
test[['aid', 'uid','score']].to_csv('submission.csv', index=False)
os.system('zip -j '+sub_zip_name+'.zip submission.csv')
os.system('mv submission.csv ./submit/'+sub_zip_name+'.csv')
os.system('mv '+sub_zip_name+'.zip ./submit/')

-1

-1

-1